# 1. Importing libraries

In [ ]:
# cuda와 pytorch의 호환을 위한 설치 버전 지정
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

In [22]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

Using cuda for training.


# 2. Loading data & Preprocessing
- 데이터 로드
  - 종별로 1,000장씩 총 7종 - 사진 7,000장
  - train_test_split 이용하여 8:2로 분할 
- 데이터 전처리

In [4]:
# 데이터 로드 및 분할
data = pd.read_csv('datasetForAWS.csv')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

for label in data['라벨'].unique():
    label_data = data[data['라벨'] == label]
    train_label_data, test_label_data = train_test_split(label_data, test_size=200, random_state=42)  # Splitting
    train_data = train_data.append(train_label_data)
    test_data = test_data.append(test_label_data)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

print(train_data[:10])

                           이미지_경로        라벨
0            img/dog/dog.4010.jpg       dog
1        img/horse/horse.382.jpeg     horse
2  img/squirrel/squirrel.176.jpeg  squirrel
3  img/squirrel/squirrel.820.jpeg  squirrel
4  img/squirrel/squirrel.892.jpeg  squirrel
5            img/cow/cow.402.jpeg       cow
6            img/cow/cow.612.jpeg       cow
7  img/elephant/elephant.828.jpeg  elephant
8         img/horse/horse.69.jpeg     horse
9    img/elephant/elephant.62.jpg  elephant


/tmp/ipykernel_32276/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
/tmp/ipykernel_32276/994303206.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
/tmp/ipykernel_32276/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
/tmp/ipykernel_32276/994303206.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
/tmp/ipykernel_32276/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [5]:
# 사용자 정의 데이터셋 클래스
class AnimalDataset(Dataset):
    def __init__(self, dataframe, label_map, transform=None):
        self.dataframe = dataframe
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        image = Image.open(image_path).convert("RGB")  # tensor 단위 오류 해결 위해 명시
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_map[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# 라벨을 정수로 매핑 - 추후 label을 숫자가 아닌 글자로 반환하도록 해주는 딕셔너리
label_map = {'cat': 0, 'cow': 1, 'dog': 2, 'elephant': 3, 'horse': 4, 'lamb': 5, 'squirrel': 6}

# 이미지 전처리를 위한 transform 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 및 데이터 로더 생성
train_dataset = AnimalDataset(train_data, label_map, transform=transform)  # 인스턴스는 init 파라미터 따름
test_dataset = AnimalDataset(test_data, label_map, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# 3. Training

In [37]:
# 모델 정의
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # imageNet 데이터로 훈련된 모델 불러옴
"""
Q. 왜 내 모델에 사용할 이미지도 아닌 imageNet 이미지로 학습한 모델을 불러오는가?
A. 이미 한 번 이미지 분류를 학습한 모델을 불러오면 복잡한 학습 과정에 들어가는 시간과 비용 절감 가능
   (가중치를 조금만 수정하면 되니까) 
"""
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7종의 동물 클래스
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습 함수
def train_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)  # GPU로 데이터 이동
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
        epoch_loss = running_loss / len(train_loader)
        tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")  # tqdm.write를 사용하여 에폭 손실 출력
    return model

In [38]:
# 모델 학습
trained_model = train_model(model, criterion, optimizer, num_epochs=20)

Epoch 1/20: 100%|██████████| 700/700 [00:37<00:00, 18.83it/s]


Epoch 1/20, Loss: 0.4001


Epoch 2/20: 100%|██████████| 700/700 [00:37<00:00, 18.85it/s]


Epoch 2/20, Loss: 0.1585


Epoch 3/20: 100%|██████████| 700/700 [00:37<00:00, 18.88it/s]


...

Epoch 18/20, Loss: 0.0391


Epoch 19/20: 100%|██████████| 700/700 [00:37<00:00, 18.81it/s]


Epoch 19/20, Loss: 0.0317


Epoch 20/20: 100%|██████████| 700/700 [00:37<00:00, 18.76it/s]

Epoch 20/20, Loss: 0.0268


# 4. Testing

In [39]:
# 테스트 함수 수정 버전
def test_model_accuracy(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # 모델을 적절한 디바이스로 이동

    with torch.no_grad():  # 그래디언트 계산을 비활성화하여 메모리 사용량을 줄이고 계산을 빠르게 함
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # 데이터를 적절한 디바이스로 이동
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'테스트 데이터에 대한 모델 정확도: {accuracy:.2f}%')

def test_model_f1_confusion_matrix(model, test_loader, device):
    model.eval()  # 모델을 평가 모드로 설정
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # 그래디언트 계산을 비활성화
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    # 혼동 행렬 계산
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    
    # F1 Score 계산
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"F1 Score: {f1:.4f}")

In [40]:
# 모델 테스트 실행
test_model_accuracy(trained_model, test_loader)  # 정확도

테스트 데이터에 대한 모델 정확도: 93.07%


In [41]:
test_model_f1_confusion_matrix(trained_model, test_loader, device)  # F1 Score & Confusion matrix

Confusion Matrix:
[[192   0   5   0   1   0   2]
 [  0 177   0   3   7  10   3]
 [  8   1 179   0   4   3   5]
 [  0   1   0 193   2   0   4]
 [  0   5   1   1 184   6   3]
 [  3   4   0   1   2 180  10]
 [  1   0   0   0   0   1 198]]
F1 Score: 0.9306


# 4. Saving Model

In [42]:
# 현재 디렉토리에 모델 저장
torch.save(model.state_dict(), 'modelv3_acc93.pth')

# 5. Recall

In [25]:
# go to modelTest.py

In [ ]:
# TODO 1 : 학습 속도 절감 (Completed)
# TODO 2 : 모델 정확도 개선 - 적절한 성능 지표, 하이퍼파라미터 등 (Completed - F1 score and confusion matrix)
# TODO 3 : 주로 영향을 미치는 하이퍼파라미터 정리 (Completed - Learning Rate, epoch 20이면 충분한 듯)